In [1]:
!pip install -U bitsandbytes accelerate

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="vilm/vinallama-7b-chat", device="cuda")

config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Hello world!<|im_end|>
<|im_start|>assistant
"""

In [ ]:
pipe(prompt)

In [22]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, TextIteratorStreamer, StoppingCriteriaList

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
# If not defined
hf_model_repo = "vilm/vinallama-7b-chat"
# device_map = 'cuda'

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    hf_model_repo,
    #   use_fast=False, # Fast tokenizer giving issues.
    padding_side="right",
)
# Load the model
model = AutoModelForCausalLM.from_pretrained(
    hf_model_repo, load_in_4bit=True, torch_dtype=torch.float16, device_map=device
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
tokenizer.is_fast  # Why is this True?

True

In [16]:
print(tokenizer.bos_token)
print(tokenizer.eos_token)
print(tokenizer.bos_token_id)

<s>
<|im_end|>
1


In [18]:
stop_token_ids = [0]


class StopOnTokens(StoppingCriteria):

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

In [19]:
# generation_config = dict(
#     temperature=0.2,
#     top_k=20,
#     top_p=0.9,
#     do_sample=True,
#     num_beams=1,
#     repetition_penalty=1.2,
#     max_new_tokens=400,
#     early_stopping=True,
# )

In [21]:
stop = StopOnTokens()
streamer = TextIteratorStreamer(
    tokenizer=tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
)

In [23]:
generation_config = dict(
    temperature=0.1,
    top_k=30,
    top_p=0.95,
    do_sample=True,
    # num_beams=1,
    repetition_penalty=1.2,
    max_new_tokens=2048,  ## 8K
    early_stopping=True,
    stopping_criteria=StoppingCriteriaList([stop]),
)

In [3]:
prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Hello world!<|im_end|>
<|im_start|>assistant
"""

In [5]:
def create_prompt(instruction: str, user_input: str) -> str:
    prompt = f"""<|im_start|>system
    {instruction}
    <|im_end|>
    <|im_start|>user
    {user_input}
    <|im_start|>assistant
    """
    return prompt

In [6]:
instruction = "Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác."
user_input = "Xin chào"

prompt = create_prompt(instruction=instruction, user_input=user_input)
prompt

'<|im_start|>system\n    Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.\n    <|im_end|>\n    <|im_start|>user\n    Xin chào\n    <|im_start|>assistant\n    '

In [25]:
token_ids = tokenizer(
    """# 1- Nền tảng công việc trước đây/hiện tại
chủ DN về xây dựng và sản xuất 15 năm
Hiện tại làm KOC cho 1 tập đoàn Mỹ có uy tín, có thị trường ở 50 quốc gia về chăm sóc sức khỏe chủ động và sắc đẹp

# 2- Thu nhập và vị trí công việc đấy ( xác định tone giọng chia sẻ là sếp thì mabg tính chuyên gia hơn)
30 triêu/tháng, đang là KOC nên muốn chia sẻ với vai trò người trãi nghiệm có kết quả và kết nối với những phụ nữ quanh 40 tuổi câu chuyện trãi nghiệm của mình và những thay đổi  tích cực về sức khỏe, vóc dáng, làn da, tâm trí, tinh thần, tư duy, thái độ, thu nhập phát triển... tất cả đều phát triển tuyệt vời trong 2 năm qua so với gần 15 làm chủ DN trước đó

# 3- Điều gì ko hài lòng hay mong muốn cải thiện ( yếu tố tạo đồng cảm với những người cùng hoàn cảnh)
Chị làm chủ doanh nghiệp xây dựng trong 15 năm nên rất đau đầu trong việc quản lý DN về thuế, công nợ thu chi, chiến lược, quản lý vốn, chi phí cố định và con người, ko có thời gian chăm sóc sức khỏe, sắc diện, làn da ngày càng bị lão hóa,khô ráp, nám và lô chân lông to, mụn, già xấu xí và ko có thời gian phát triển bản thân và chăm sóc cho con nhất là khi con vào tuổi dậy thì với nhiều biến đổi tâm sinh lý, thường xuyên tranh cải với chồng do ko cùng quan điểm trong giải quyết công việc, cũng như cảm thấy bức bối như bị giam cầm trong công ty để giải quyết đống sổ sách 8-10 tiếng/ngày.  Chị quá mệt mõi sau 15 năm cùng chồng điều hành công ty riêng nên đã nghỉ việc ở nhà chăm sóc gia đình, làm nội trợ toàn thời gian trong 3 năm.  Sau dịch bệnh, chị đã luôn tìm kiếm 1 cơ hội, công việc mới để phụ giúp kinh tế cho chồng, khắc phục những khó khăn của chủ DN xây dựng ở trên, phù hợp với mong muốn có nguồn thu nhập tốt để tự do, lo cho gia đình chu đáo, không phụ thuộc thu nhập của chồng, có cơ hội học hỏi để phát triển bản thân và linh động thời gian làm việc để được tự nấu ăn, đưa đón và trò chuyện với con, đi du lịch trãi nghiệm khắp nơi cùng gia đình mình.

# 4- Đến với câu chuyện KOC wellness trong hoàn cảnh nào, đã bắt đầu ra sao
Chị trước đây là 1 chủ DN về xây dựng trong 15 năm.  Công việc ấy mang lại cho chị tài chính khá tốt, có nhà, xe và cho con cái ăn học tới nơi tới chốn, trãi nghiệm nhìu nơi cùng với nhau.  Tuy nhiên chị lại rất đau đầu trong việc quản lý DN về thuế, công nợ thu chi, chiến lược, quản lý vốn, chi phí cố định và con người, ko có thời gian chăm sóc sức khỏe, sắc diện, làn da ngày càng bị lão hóa,khô ráp, nám và lô chân lông to, mụn và phát triển bản thân và chăm sóc cho con nhất là khi con vào tuổi dậy thì với nhiều biến đổi tâm sinh lý, thường xuyên tranh cải với chồng do ko cùng quan điểm trong giải quyết công việc, cũng như cảm thấy bức bối như bị giam cầm trong công ty giải quyết đống sổ sách 8-10 tiếng/ngày.  Chị quá mệt mõi sau 15 năm cùng chồng điều hành công ty riêng nên đã nghỉ việc ở nhà chăm sóc gia đình, làm nội trợ toàn thời gian trong 3 năm.  Sau dịch bệnh, chị đã luôn tìm kiếm 1 cơ hội, công việc mới để phụ giúp kinh tế cho chồng, khắc phục những khó khăn của chủ DN xây dựng ở trên, phù hợp với mong muốn có nguồn thu nhập tốt để tự do, lo cho gia đình chu đáo, không phụ thuộc chồng, có cơ hội học hỏi để phát triển bản thân và linh động thời gian làm việc để được tự nấu ăn, đưa đón và trò chuyện với con, đi du lịch trãi nghiệm khắp nơi cùng gia đình mình.  Chị đã thử nộp đơn xin việc làm tư vấn viên cho trung tâm ngoại ngữ nhưng sau đó rút đơn vì nhận ra nó cũng ko thể đáp ứng các mong muốn trên của chị.  Chị có 1 người bạn là chủ DN tại TPHCM đã chia sẻ cho chị cơ hội làm KOC trong lĩnh vực sức khỏe và sắc đẹp cho tập đoàn Mỹ uy tín, có thị trường ở 50 quốc gia.. Chị ấy sử dụng mạng xã hội 3-4h/ngày để làm việc và mở rộng lượng khách hàng, càng kinh doanh càng thấy trẻ khỏe ra.  Thế nên chị quyết định thay đổi, trở thành 1 KOC.  Chị được mentor của mình nhắc nhở rằng “máy bay muốn cất cánh cần phải tăng tốc thật nhanh, khi đạt được tốc độ cho phép nó mới rời đường băng   bay lên bầu trời được” nên chị đã nổ lực để cất cánh để đạt được ước mơ của mình ở tuổi 45.  Giờ đây, sau 2 năm làm KOC, chị đã chuyển đổi thành 1 người phụ nữ trẻ khỏe so với tuổi mình, xinh đẹp hơn, tràn đầy năng lượng tích cực, tự tin chia sẻ hành trình trãi nghiệm và thay đổi của mình trong 2 năm qua với mọi người, thu nhập ngày càng tốt hơn từ rất  nhiều quốc gia, được học hỏi cập nhật kiến thức và kỷ năng mới liên tục từ những người thầy rất giỏi đến từ nhiều nước và đã thành công, tự tay chăm sóc gia đình mình và luôn ở bên chồng con, cha mẹ mỗi khi họ cần, được đi du lịch đó đây với nhưng chuyến trip tưởng thưởng tiêu chuẩn 5 sao của tập đoàn, sống độc lập, có giá trị và cảm thấy rất hạnh phúc.      

# 5- Thích gì, hướng đến mục tiêu gì
Hướng đến cuộc sống giàu có thịnh vượng về tài chánh, sức khỏe, thời gian, tự do trong phong cách sống, theo đuổi cuộc sống trẻ khỏe và giúp đỡ 1000 chị em được sống như mình, yêu thương bản thân, độc lập về tài chính, quan tâm tới giáo dục trẻ em, chăm sóc người già, yêu thích thiên nhiên

# 6- 3-5 từ miêu tả tính cách
tích cực, năng động, ấm áp, tinh cậy, nhẹ nhàng, quyết liệt

#7- Thành tựu hiện tại: 
cuộc sống hạnh phúc với chồng con, các con chia sẻ với ba mẹ mọi buồn vui, cả nhà trẻ, khỏe và năng lượng tích cực, là nguồn cảm hứng cho nhiều gia đình về lối sống của mình

# 8- Sở thích
thích vận động, âm nhạc, nhiều màu sắc tươi vui, thích du lịch, đồng hành cùng chồng con mọi nơi, thích học hỏi để phát triển bản thân, thích thiên nhiên, rừng cây và lối sống xanh

# 9- Hình tượng muốn trở thành khi chia sẻ mxh
Người phụ nữ ấm áp, trẻ khỏe, năng động, hiện đại, sống tích cực và thích viết lách, chia sẻ trãi nghiệm của mình về những thứ tốt đẹp nhận được cũng như những vấp ngã, sai lầm đã qua về gia đình, về giáo dục và chăm sóc sức khỏe, sắc diện

# 10- Tên cá nhân, giới tính và độ tuổi để dùng trong script hoặc content khi cần
Mình là Việt Ngân, 47 tuổi, Nữ
"""
)["input_ids"]

In [41]:
token_ids_1 = tokenizer(
    """# 1- Nền tảng công việc trước đây/hiện tại: chủ DN về xây dựng và sản xuất 15 năm
Hiện tại làm KOC cho 1 tập đoàn Mỹ có uy tín, có thị trường ở 50 quốc gia về chăm sóc sức khỏe chủ động và sắc đẹp
# 2- Thu nhập và vị trí công việc đấy ( xác định tone giọng chia sẻ là sếp thì mabg tính chuyên gia hơn): 30 triêu/tháng, đang là KOC nên
"""
)["input_ids"]

token_ids_2 = tokenizer(
    """# 1- Nền tảng công việc trước đây/hiện tại: chủ DN về xây dựng và sản xuất 15 năm
Hiện tại làm KOC cho 1 tập đoàn Mỹ có uy tín, có thị trường ở 50 quốc gia về chăm sóc sức khỏe chủ động và sắc đẹp



# 2- Thu nhập và vị trí công việc đấy ( xác định tone giọng chia sẻ là sếp thì mabg tính chuyên gia hơn)
30 triêu/tháng, đang là KOC nên
"""
)["input_ids"]

In [42]:
len(token_ids_1), len(token_ids_2)

(96, 100)

In [43]:
vocab_mapping = tokenizer.get_vocab()

In [44]:
vocab_mapping

{'▁Maria': 6217,
 'backgroundColor': 29605,
 '▁Houston': 24327,
 'Errors': 22463,
 'Vĩnh': 38933,
 '());': 3310,
 'istrative': 40119,
 'Vua': 42242,
 '▁Music': 6125,
 'коно': 13335,
 '▁Latvia': 44884,
 '▁Р': 1483,
 '▁Building': 17166,
 'Cadn': 40093,
 'Argument': 15730,
 'UE': 4462,
 '▁Yo': 34803,
 'itute': 12356,
 'groupby': 27789,
 'iago': 37955,
 '▁Vy': 35223,
 '▁Album': 10217,
 '▁passwords': 27630,
 'bern': 5892,
 '41': 35949,
 'td': 1594,
 'CV': 15633,
 '▁Nikol': 22958,
 '你': 30919,
 '<0x8A>': 141,
 '▁nhện': 36429,
 '▁ю': 8053,
 '▁competed': 22071,
 '▁cons': 1136,
 'ocz': 19393,
 '▁painting': 20413,
 'ані': 15264,
 '▁CURL': 24222,
 '▁Tw': 8168,
 '▁Nep': 22806,
 '현': 31680,
 'Ant': 13448,
 '▁cứu': 32548,
 '▁2012.': 34708,
 'К': 30014,
 '▁biếc': 43312,
 '▁spacing': 29250,
 '▁SY': 28962,
 '▁Brook': 18737,
 '▁eq': 11594,
 'cao': 37120,
 '▁2.100': 44745,
 'blatt': 22801,
 'ansk': 12190,
 '▁tầng': 33138,
 'ு': 30837,
 'theo': 34931,
 '▁március': 24605,
 'olean': 4132,
 '▁Según': 24109,


In [27]:
tokenizer.decode(token_ids)

'<s> # 1- Nền tảng công việc trước đây/hiện tại\nchủ DN về xây dựng và sản xuất 15 năm\nHiện tại làm KOC cho 1 tập đoàn Mỹ có uy tín, có thị trường ở 50 quốc gia về chăm sóc sức khỏe chủ động và sắc đẹp\n\n# 2- Thu nhập và vị trí công việc đấy ( xác định tone giọng chia sẻ là sếp thì mabg tính chuyên gia hơn)\n30 triêu/tháng, đang là KOC nên muốn chia sẻ với vai trò người trãi nghiệm có kết quả và kết nối với những phụ nữ quanh 40 tuổi câu chuyện trãi nghiệm của mình và những thay đổi  tích cực về sức khỏe, vóc dáng, làn da, tâm trí, tinh thần, tư duy, thái độ, thu nhập phát triển... tất cả đều phát triển tuyệt vời trong 2 năm qua so với gần 15 làm chủ DN trước đó\n\n# 3- Điều gì ko hài lòng hay mong muốn cải thiện ( yếu tố tạo đồng cảm với những người cùng hoàn cảnh)\nChị làm chủ doanh nghiệp xây dựng trong 15 năm nên rất đau đầu trong việc quản lý DN về thuế, công nợ thu chi, chiến lược, quản lý vốn, chi phí cố định và con người, ko có thời gian chăm sóc sức khỏe, sắc diện, làn da ng

In [11]:
prompt

'<|im_start|>system\n    Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.\n    <|im_end|>\n    <|im_start|>user\n    Xin chào\n    <|im_start|>assistant\n    '

In [24]:
# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)  # .input_ids.cuda()

# Run the model to infere an output
# outputs = model.generate(
#     input_ids=input_ids,
#     max_new_tokens=2048,
#     do_sample=True,
#     top_p=0.9,
#     temperature=0.3
# )
generation_output = model.generate(
    input_ids=inputs["input_ids"].to(device),
    attention_mask=inputs["attention_mask"].to(device),
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    **generation_config,
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/congvm/miniconda3/envs/dl-py3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/home/congvm/miniconda3/envs/dl-py3/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


In [25]:
# generation_output_ = model.generate(
#     input_ids=inputs["input_ids"].to(device), **generation_config
# )

In [26]:
s = generation_output[0]
output = tokenizer.decode(s, skip_special_tokens=True)

In [28]:
output

'<|im_start|> system\n    Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.\n     \n    <|im_start|> user\n    Xin chào\n    <|im_start|> assistant\n    '

In [29]:
# outputs_str = (
#     tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
#     .split("assistant\n")[-1]
#     .strip()
# )
# outputs_str

In [30]:
# print(
#     f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}"
# )

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model_name = "VietnamAIHub/Vietnamese_LLama2_13B_8K_SFT_General_Domain_Knowledge"

## Loading Base LLaMa model weight and Merge with Adapter Weight wiht the base model
m = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    pretraining_tp=1,
    # use_auth_token=True,
    # trust_remote_code=True,
    # cache_dir=cache_dir,
)

tok = AutoTokenizer.from_pretrained(
    model_name,
    # cache_dir=cache_dir,
    padding_side="right",
    use_fast=False,  # Fast tokenizer giving issues.
    tokenizer_type="llama",  # if 'llama' in args.model_name_or_path else None, # Needed for HF name change
    use_auth_token=True,
)
tok.bos_token_id = 1
stop_token_ids = [0]


class StopOnTokens(StoppingCriteria):

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False


generation_config = dict(
    temperature=0.2,
    top_k=20,
    top_p=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.2,
    max_new_tokens=400,
    early_stopping=True,
)

prompts_input = "Cách để học tập về một môn học thật tốt"
system_prompt = f"<s>[INST] <<SYS>>\n You are a helpful assistant, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your \
        answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure\
        that your responses are socially unbiased and positive in nature.\
        If a question does not make any sense, or is not factually coherent, explain why instead of answering something not \
        correct. If you don't know the answer to a question, please response as language model you are not able to respone detailed to these kind of question.\n<</SYS>>\n\n {prompts_input} [/INST] "


input_ids = tok(message, return_tensors="pt").input_ids
input_ids = input_ids.to(m.device)
stop = StopOnTokens()
streamer = TextIteratorStreamer(
    tok, timeout=10.0, skip_prompt=True, skip_special_tokens=True
)

# #print(tok.decode(output[0]))
generation_config = dict(
    temperature=0.1,
    top_k=30,
    top_p=0.95,
    do_sample=True,
    # num_beams=1,
    repetition_penalty=1.2,
    max_new_tokens=2048,  ## 8K
    early_stopping=True,
    stopping_criteria=StoppingCriteriaList([stop]),
)
inputs = tok(message, return_tensors="pt")  # add_special_tokens=False ?
generation_output = m.generate(
    input_ids=inputs["input_ids"].to(device),
    attention_mask=inputs["attention_mask"].to(device),
    eos_token_id=tok.eos_token_id,
    pad_token_id=tok.pad_token_id,
    **generation_config,
)
generation_output_ = m.generate(
    input_ids=inputs["input_ids"].to(device), **generation_config
)

s = generation_output[0]
output = tok.decode(s, skip_special_tokens=True)
# response = output.split("### Output:")[1].strip()
print(output)